In [2]:
!pip install pybit

  Using cached pybit-5.8.0-py2.py3-none-any.whl


In [2]:
from pybit.unified_trading import HTTP
import pandas as pd
from datetime import datetime, timedelta

# Khởi tạo phiên làm việc với testnet
session = HTTP(testnet=True)

# Định nghĩa các tham số để lấy dữ liệu
category = "inverse"
symbols = ['BTCUSD', 'AVAXUSDT', 'AXSUSDT', 'BNBUSDT']  # Danh sách các cặp tiền
interval = 60  # khoảng thời gian 60 phút

# Đặt thời gian bắt đầu và kết thúc
start_date = "2024-01-01"
end_date = "2024-06-01"

# Chuyển đổi thời gian sang định dạng milliseconds
start_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)  # milliseconds
end_timestamp = int(pd.Timestamp(end_date).timestamp() * 1000)  # milliseconds

# Số milliseconds trong 1 tuần
one_week_ms = 7 * 24 * 60 * 60 * 1000

# Khởi tạo danh sách để chứa tất cả các dữ liệu nến từ các cặp tiền
all_data = []

# Lặp qua từng cặp tiền trong danh sách
for symbol in symbols:
    print(f"Đang lấy dữ liệu cho cặp tiền: {symbol}")
    
    # Khởi tạo danh sách để chứa dữ liệu nến cho từng cặp tiền
    all_candles = []
    
    # Lặp qua từng khoảng thời gian 1 tuần và lấy dữ liệu
    current_start = start_timestamp
    while current_start < end_timestamp:
        current_end = min(current_start + one_week_ms, end_timestamp)
        
        # Lấy dữ liệu nến từ API
        response = session.get_kline(
            category=category,
            symbol=symbol,
            interval=interval,
            start=current_start,
            end=current_end,
        )
        
        # Kiểm tra phản hồi và xử lý dữ liệu
        if response['retCode'] == 0 and 'list' in response['result']:
            candles = response['result']['list']
            all_candles.extend(candles)  # Thêm dữ liệu vào danh sách chung
        else:
            print(f"Không có dữ liệu nến hoặc có lỗi trong phản hồi cho khoảng {pd.to_datetime(current_start, unit='ms')} đến {pd.to_datetime(current_end, unit='ms')} cho cặp {symbol}.")
        
        # Cập nhật thời gian bắt đầu cho tuần tiếp theo
        current_start = current_end
    
    # Chuyển đổi dữ liệu thành DataFrame nếu có dữ liệu
    if all_candles:
        df = pd.DataFrame(all_candles, columns=['Time (UTC+3)', 'Open Price', 'Highest Price', 'Lowest Price', 'Close Price', 'Trade Volume', 'Another Value'])
        # Chuyển đổi cột thời gian từ milliseconds sang định dạng datetime
        df['Time (UTC+3)'] = pd.to_datetime(df['Time (UTC+3)'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-3')
        # Định dạng lại cột thời gian theo mẫu "YYYY.MM.DD HH:MM"
        df['Time (UTC+3)'] = df['Time (UTC+3)'].dt.strftime('%Y.%m.%d %H:%M')

        # Chọn cột cần thiết
        df = df[['Time (UTC+3)', 'Open Price', 'Highest Price', 'Lowest Price', 'Close Price', 'Trade Volume']]

        # Đổi kiểu dữ liệu cho các cột số
        df[['Open Price', 'Highest Price', 'Lowest Price', 'Close Price', 'Trade Volume']] = df[['Open Price', 'Highest Price', 'Lowest Price', 'Close Price', 'Trade Volume']].astype(float)

        # Thêm cột 'Symbol' để đánh dấu cặp tiền cho dữ liệu này
        df['Symbol'] = symbol
        
        # Thêm dữ liệu vào danh sách chung
        all_data.append(df)
    else:
        print(f"Không có dữ liệu nến nào cho cặp tiền {symbol} trong khoảng thời gian yêu cầu.")

# Kết hợp tất cả dữ liệu từ các cặp tiền thành một DataFrame lớn
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Hiển thị DataFrame kết hợp
    print(combined_df)
else:
    print("Không có dữ liệu nến nào cho tất cả các cặp tiền trong khoảng thời gian yêu cầu.")


Đang lấy dữ liệu cho cặp tiền: BTCUSD


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_30840\1864293240.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Time (UTC+3)'] = pd.to_datetime(df['Time (UTC+3)'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-3')


Đang lấy dữ liệu cho cặp tiền: AVAXUSDT


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_30840\1864293240.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Time (UTC+3)'] = pd.to_datetime(df['Time (UTC+3)'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-3')


Đang lấy dữ liệu cho cặp tiền: AXSUSDT


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_30840\1864293240.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Time (UTC+3)'] = pd.to_datetime(df['Time (UTC+3)'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-3')


Đang lấy dữ liệu cho cặp tiền: BNBUSDT


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_30840\1864293240.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Time (UTC+3)'] = pd.to_datetime(df['Time (UTC+3)'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-3')


           Time (UTC+3)  Open Price  Highest Price  Lowest Price  Close Price  \
0      2024.01.08 02:59     43946.0        44003.0       43656.5      43656.5   
1      2024.01.08 02:00     43893.5        44046.0       43636.5      43946.0   
2      2024.01.08 00:59     44259.5        44259.5       43813.0      43893.5   
3      2024.01.08 00:00     44147.5        44272.0       44146.5      44259.5   
4      2024.01.07 22:59     44262.0        44292.0       44147.5      44147.5   
...                 ...         ...            ...           ...          ...   
14675  2024.05.27 06:59       605.5          606.5         604.0        604.0   
14676  2024.05.27 06:00       605.5          607.0         604.5        605.5   
14677  2024.05.27 04:59       603.5          606.5         603.0        605.5   
14678  2024.05.27 04:00       601.5          604.5         600.5        603.5   
14679  2024.05.27 02:59       600.0          602.5         599.5        601.5   

       Trade Volume   Symbo